In [117]:
import pandas as pd

In [118]:
from datetime import *

In [119]:
df = pd.read_csv('us-states.csv')

In [120]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

In [121]:
dictList = {key:{} for key in states}

In [122]:
for state in states:
    stateFilter = (df['state']==state)
    dictList[state]= df[stateFilter]
    dictList[state].reset_index(inplace=True)

In [123]:
def changeDate(aDate):
    aDate = datetime.strptime(aDate, '%Y-%m-%d')
    finalDate = aDate.date()
    return finalDate

In [124]:
def getFirstEvent(name):
    result = dictList[name].loc[0, 'date']
    return result

In [125]:
# This function returns the number of days between the first case and the first death
def caseToDeath(state):
    minList = []
    # We want to find the date of the first death so it has to be greater than 0, if we append it to a list,
    # then we can get the first value and that will be the date of the first death
    for ind, row in dictList[state].iterrows():
        if row[5] > 0:
            minList.append(row[1])
    firstDeath = minList[0]
    firstCaseDate = getFirstEvent(state)
    # change the dates to datetime objects to be able to get difference between first death date and first case date
    firstDeathDate = changeDate(firstDeath)
    firstCaseDate = changeDate(firstCaseDate)
    daysInBetween = firstDeathDate - firstCaseDate
    daysInBetween = daysInBetween.days

    return daysInBetween

In [126]:
def eventsOnDate(state, date,event):
    dictList[state].reset_index(inplace=True)
    dictList[state].set_index('date',inplace=True)
    if event == 'cases':
        result = dictList[state].loc[date,'cases']
    elif event == 'deaths':
        result = dictList[state].loc[date,'deaths']
    return result

In [127]:
print(eventsOnDate("Iowa","2020-03-20","cases"))

45


In [128]:
def casesBetweenDates(state,firstDate,secondDate):
    firstResults = eventsOnDate(state, firstDate,'cases')
    secondResults = eventsOnDate(state, secondDate,'cases')
    firstResults = int(firstResults)
    secondResults = int(secondResults)
    difference = secondResults - firstResults
    return difference

In [129]:
print(casesBetweenDates('Iowa','2020-03-20','2020-03-21'))

23


In [130]:
lastInd = dictList['Arkansas'].index[-1]

In [131]:
print(lastInd)

96


In [132]:
df_Ark = dictList['Arkansas']

In [133]:
df_Ark.loc[lastInd,'date']

'2020-06-15'

In [134]:
finalD = df_Ark.loc[lastInd,'date']

In [135]:
print(type(finalD))

<type 'str'>


In [136]:
def previousDate(date):
    date = changeDate(date)
    dateBefore = date - timedelta(days=1)
    return dateBefore

In [137]:
print(previousDate(finalD))

2020-06-14


In [138]:
casesList = {key: [] for key in states}

In [139]:
stateDataF = {key:{} for key in states}

In [140]:
def caseChanges(state):
    
    # make it easier to access
    stateDF = dictList[state]
    # Will be used for indexing
    i = 1
    
    stateDF.reset_index(inplace=True)
    
    otherLastIndex = stateDF.index[-1]
    
    for item in range(otherLastIndex,0,-1):
        
        # The last index
        lastIndex = stateDF.index[-i]
        
        # Date that corresponds to the last index
        lastDate = stateDF.loc[lastIndex,'date']
        
        # Number of casese that correspond to the last date
        lastDateCases = stateDF.loc[lastIndex,'cases']
        
        # Date for the day before
        dayBefore = stateDF.loc[lastIndex-1,'date']
        
         # The cases for the day before the last index
        previousDayCases = stateDF.loc[(lastIndex-1),'cases']
        
        # Change between the previous day cases and the last date cases
        difference = lastDateCases - previousDayCases
        dateString = str(dayBefore) + ' to ' + str(lastDate)
        
        # Add to the list that will be transformed into a DataFrame
        casesList[state].append([lastDate,lastDateCases,dayBefore,previousDayCases,difference])
        #casesList[state].append(dayBefore)
        i+=1
    casesList[state].reverse()
    
     
    return difference

In [141]:
for name in states:
    caseChanges(name)

In [142]:
for stateName in states:
    stateDataF[stateName]=pd.DataFrame(casesList[stateName],columns=['Date','Total Cases','Previous Date','Cases','Daily Change'])

In [143]:
stateDataF['Florida']

,Date,Total Cases,Previous Date,Cases,Daily Change
0,2020-03-02,2,2020-03-01,2,0
1,2020-03-03,3,2020-03-02,2,1
2,2020-03-04,3,2020-03-03,3,0
3,2020-03-05,4,2020-03-04,3,1
4,2020-03-06,7,2020-03-05,4,3
5,2020-03-07,12,2020-03-06,7,5
6,2020-03-08,13,2020-03-07,12,1
7,2020-03-09,14,2020-03-08,13,1
8,2020-03-10,23,2020-03-09,14,9
9,2020-03-11,26,2020-03-10,23,3
